# Homework 3

## Due back by midnight on Wednesday, April 25th.


**Names of Collaborators:** 

**Wedsites Used: **



### Networkx; Pygraphviz

We will be using `networkx` and `pygraphviz` in some of the exercises below. Knowing the format/representation of the results that you obtain from algorithms/constructs in the API is critical, so that you can modify or tailor the code accordingly.

**Important:** Make sure that you install all the packages needed to make the following imports work correctly:


In [3]:
import pydot
import pygraphviz
import matplotlib 
% matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx

ImportError: No module named pygraphviz

## Problem 1: Conversion to Leftist Trees

Suppose you are given a binary search trees $T_1$. We know that a tree rotation preserves the search tree property, and a natural question to ask is whether it is possible to start with $T_1$ and through a series of appropriate rotations, convert it to a known structure. In this problem, we will figure out how to convert it to a canonical structure for a binary search tree, a **leftist tree**.

To be precise, we will assume that by **node**, we always mean a key-bearing node: leaf nodes have no child nodes whereas internal nodes have either one or two child nodes: missing child nodes are marked as null (aka `None`).

The following code provides the basic class definition of a BST node and utility functions to create a BST and a dot representation of the tree (after wrapping this within `graph {` and `}`, you can run the `dot` program on it to create a figure of the tree). You can use the figure to see how the trees look after rotations etc.

In [ ]:
## Do not modify!!

class BSTNode:
    def __init__(self, key, left=None, right=None, parent=None):
        """Create a leaf node"""
        self._key = key
        self._left = left
        self._right = right
        self._parent = parent
        
    def __repr__(self):
        return str(self._key)

def makeTree(preorder):
    """Returns a reference to the root of a binary search tree
    
    preorder: preorder sequence of labels in the BST
    """
    inorder = ''.join(sorted(preorder))
    if len(preorder)==0 and preorder==inorder:
        return None
    elif len(preorder)==1 and preorder==inorder:
        return BSTNode(preorder)
    else:
        rootLabel = preorder[0]
        ix = inorder.index(rootLabel)
        left_st = makeTree(preorder[1:1+ix])
        right_st = makeTree(preorder[1+ix:])
        node = BSTNode(rootLabel, left_st, right_st, None)
        if left_st:
            left_st._parent = node
        if right_st:
            right_st._parent = node
        return node

In [ ]:
## Do not modify!!

tmp = 0
def drawTree(node):
    """Returns a graphviz representation (string) of the tree
    rooted at node
    
    node: a BST node
    """
    def hiddenEdge(rootLabel):
        global tmp
        stLabel = 'x'+str(tmp)
        tmp += 1
        s = "{0} [style=invis]\n".format(stLabel)
        s += "{0} -> {1} [style=invis];\n".format(rootLabel, stLabel)
        return s
    s = ''   
    if not(node._left) and not(node._right): # leaf node
        return s
    if node._left:
        s1 = "{0} -> {1};\n".format(node._key, node._left._key)
        s1 += drawTree(node._left)
        s += s1
    else:
        s += hiddenEdge(node._key)
    if node._right:
        s2 = "{0} -> {1};\n".format(node._key, node._right._key)
        s2 += drawTree(node._right)
        s += s2
    else:
        s += hiddenEdge(node._key)
    return s
    

In [ ]:
## Do not modify!! Meant for testing

t = makeTree(preorder="EDBACHFGIJ")  # E is the root!
print(drawTree(t))

drawing = open("tree.dot", 'w')
drawing.write('digraph { \n' + drawTree(t) + '\n}\n')
drawing.close()

t1= nx.DiGraph(nx.drawing.nx_agraph.read_dot('tree.dot'))
pos=nx.drawing.nx_agraph.graphviz_layout(t1, prog='dot')
nx.draw(t1, pos, with_labels=True, arrows=True) 


### The conversion algorithm

Call a tree **leftist** if it is a **chain of nodes** starting from the root node and ending at a single leaf node such that **every** internal node has exactly one left child. To convert an arbitrary binary search tree to a leftist tree, consider the following recursive algorithm. Repeatedly perform a left rotation at the root node until it has no right child. Then recursively apply the same algorithm to the left subtree of the root node (figure out the base case for the recursion).

Complete the code below. First program the `rotateLeftAt` function. Then complete the `leftistTree` function that takes an arbitrary BST as argument and returns the BST representation of the corresponding leftist tree.

In [ ]:
## Complete these functions!!

def rotateLeftAt(bst):
    """Returns a reference to the root node of the subtree
    that results from performing a left rotation at the root of bst
    
    bst: the root of a binary search (sub)tree that has a right child
    """
    return bst
    
    
def leftistTree(bst):
    """Convert bst to return a reference to the root node of 
    corresponding leftist tree

    bst: root node of the bst (not null)
    """
    return bst


In [ ]:
## Do not modify!! This code is meant for use with tests.

def isLeftist(t):
    """Checks if t is a leftist bst"""
    if not(t._left) and not(t._right):
        return True
    if t._right:
        return False
    return isLeftist(t._left)

t2 = leftistTree(t)
isLeftist(t2)  # should return True!

In [ ]:
## Do not modify!! This code is meant for testing
drawing = open("leftist.dot", 'w')
drawing.write('digraph { \n' + drawTree(t2) + '\n}\n')
drawing.close()
leftist = nx.DiGraph(nx.drawing.nx_agraph.read_dot('leftist.dot'))
pos=nx.drawing.nx_agraph.graphviz_layout(leftist, prog='dot')
nx.draw(leftist, pos, node_color='#A0CBE2', with_labels=True, arrows=True)

## Problem 2

Epidimeologists frequently use graphs to model the spread of disease. In one such simplified framework, they get lots of data of the form: 
$$[p_i, p_j, t_1]$$
indicating that person $p_i$ had contact with person $p_j$ at time $t_1$. Infections can be traced in a simple manner: if $[p_i, p_j, t_1]$ and $[p_j, p_k, t_2]$ are both present in the data for $t_1 \leq t_2$, then we can assume that if $p_i$ was infected at time $t_1$, then  $p_j$ would get the infection and further pass it to $p_k$ at a later time $t_2$, and so on through a chain of infections. Note that this model assumes that a person once infected, stays infected forever.

Provide a **clear, detailed** description of how you would model this as a graph (so you need to indicate what the vertices are, and what the edges are), and design an algorithm that can answer questions such as: given a collection of $m$ triples of data in the form above with a total of $n$ persons amomg the triples, decide whether a person $p$ infected at time $t_x$ could have infected another given person $q$ by time $t_y \geq t_x$. Your algorithm should run in time proportional to $O(m+n)$: you should explain this. 




#### Written Answer for Problem 2

In this problem, we can model the data as directed graph where n vertices are $p_i$ and m edges are the links between $p_i$ and $p_j$ and weight is $t_k$. example : {[p1,p2,1],[p2,p3,2],[p2,p4,3],[p3,p4,5]} <img src="graph1.png" width="500"/>.
To answer the query with time complexity -- $O(m+n)$,  we should construct the algorithm as follows : 


After generating the graph we need to begin our traversal from the person p, we do a dfs or bfs traversal but with some constraints to propagate and select edges to explore, the visited state is applied to edges and not vertices because we might visit a node many times but we should not revisit same edge because it has the same value as before and no information or impact is added to the overall state of infections in this case.

1. initiate an array Affected[n] of size n with infinite value
2. Select node p and initialize it with $t_x$ 
3. Visit every node affected by p if the edge is >= $t_x$ and update the values of the array Affected with the corresponding weights/time if and only if it's less than previous value (the initial state is inifinite )
4. for every node affected/updated we do recursively the same process, the Affected array update constraint is keeping the algorithm out of infinite loops.
5. when the recursive process stops, we have the final "Affected" array updated with the values of first infections of the person, so to answer the query we look for the numerical value Affected[$q$] and if it's less or equal than t_y then we have a TRUE, otherwise it's FALSE.

Complexity discussion: 
Given the Update only in case of lower affect time, the propagation won't produce any loops in the recursivity so it's comparable to a classic bfs traversal and at the worst case the algorithm will pass through all the edges and that makes it $O(m+n)$.


## Problem 3

Consider the graph shown below as a figure:

<img src="example.png" width="500"/>

(a) Construct a dot file encoding the graph and call it `p3.dot` - you should submit the file along with this notebook as an attachment.

In [ ]:
## Complete code for 3(a) here

(b) Use the networkx API to compute a depth-first search tree rooted at node 1. Print out the tree edges, the back edges, the forward edges and the cross edges computed in the search (you will need to understand the API to do this).

In [ ]:
## Complete code for 3(b) here

(c) Use the networkx API to compute and print the strongly connected components (SCCs) of the graph. Your code should be able to automatically construct the SCC super-graph (you should not do this by hand for the specific graph above!) and display it **inline (within the notebook)** using matplotlib and pygraphviz as illustrated above.

You should use the super-vertex label the concatenation of the string labels of the constitutent nodes of the original graph separated by underscores (store the list of nodes in a super-vertex as an attribute of the super-node in the networkx representation). For example, the labels of the super-nodes corresponding to the graph shown above could be "1_3", "0", "2" and "4_5_6_7".


In [ ]:
## Complete code for 3(c)

## Problem 4

A **proper coloring** of a graph is defined in Definition 5.8.1 on the following webpage:

**https://www.whitman.edu/mathematics/cgt_online/book/section05.08.html**

Basically, in a proper coloring, no two adjacent vertices have the same color. 

A **planar graph** is one that can be **drawn on a plane (i.e. a sheet of paper) without any of crossing edges**. The following webpage 

**https://www.whitman.edu/mathematics/cgt_online/book/section05.10.html**

provides basic definitions of planarity and establishes some of the basic properties of planar graphs (e.g. Euler's formula, upper bounds on the number of edges of a planar graph) and culmimates in a nice proof that shows that every planar graph can be colored with **at most 5 colors** (Theorem 5.10.6 at the bottom of the page).

Of particular interest to us is Lemma 5.10.5 on the same webpage that asserts that in every planar graph, there must be a vertex of degree at most 5. This immediately implies the following weaker result: **every planar graph can be colored with at most 6 colors**. The algorithm is straightforward:

- In the graph, find a vertex (call it $v$) whose degree is at most 5. \

- Consider the **rest of the graph, $G-v$**, obtained by deleting $v$ and its incident edges. This graph is also planar by definition, and so we can recursively, compute a 6-coloring for it.

- Now, $v$'s neighbors in $G$ are all colored, and between them, only have at most 5 colors. At east one color remains from the set of six colors that has not been used on $v$'s neighboirs. Color $v$ with any such color.

The resulting coloring is a 6-coloring!



#### Data

We will use the supplied file, `neighbors-states.csv`, that contains information about US states and their neighboring states. For example, the file has lines

>NJ,NY

and

>NJ,PA

that show that New Jersey has Pennsylvania and New York as its 
neighbors. The resulting graph will naturally be a planar graph.

### 4(a) 

Write a function that reads `neighbors-states.csv` (or any file like it that describes edges, one per line) and returns an undirected networkx graph.

In [ ]:
## Complete thos code

def make_graph(file):
    """Returns a networkx undirected graph
    
    file: a csv specification of a graph
    """
    return nx.Graph()

#### Selecting colors

Here is an example of how `networkx` and `matplotlib` assign colors to nodes of graphs.

In [ ]:
color_list = ['red', 'purple', 'green', 'orange', 'blue', 'yellow']*3
t1.nodes()

In [ ]:
len(t1.nodes())

In [ ]:
pos=nx.drawing.nx_agraph.graphviz_layout(t1, prog='dot')
nx.draw(t1, pos, node_color=color_list[:len(t1.nodes())],
        with_labels=True, arrows=True)

So, colors are assigned in the same order as the listing of nodes of the graph (via the `nodes` method).

### 4(b) 6-coloring Planar Graphs

Implement the algorithm that was described earlier: given a planar graph as input (in `networkx` undirected graph representation), your code should be able to construct an assignment of colors to the nodes that is a proper coloring with at most six colors. For uniformity, please use `color_list` above as your list of legal colors. 

**Hint:** You should not have to "delete" edges and vertices: simply keep track of a list of eliminated vertices, and the "current" degrees of surviving vertices in a dictionary. When there are 6 (or fewer) surviving vertices in the graph, you can simply color them with 6 colors. 


In [ ]:
## Implement the function below: the stub code does niot always return
## a correct 6-coloring

def six_coloring(g):
    """Returns a 6-coloring of g as a list of colors
    
    g: networkx undirected graph known to be planar
    """
    n = len(g.nodes())//6 + 1
    color_list = ['red', 'purple', 'green', 'orange', 'blue', 'yellow']*n
    color_list = color_list[:len(g.nodes())]
    return color_list